In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc, average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import json
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from os import path
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from xgboost import XGBClassifier
from itertools import starmap
from msgspec import Struct, field as mfield
from msgspec.json import decode

In [ ]:
!pip install msgspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.3/210.3 kB 1.6 MB/s eta 0:00:00


In [ ]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 5.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
cd /content/drive/MyDrive/Matching

/content/drive/MyDrive/Matching


In [ ]:
#Загрузка  и соединение всех данных в один датафрейм (модицифированные функции из бейзлайна)
def load_data(attributes_path, resnet_path, text_and_bert_path, train_path):
    attributes = pd.read_parquet(attributes_path)
    resnet = pd.read_parquet(resnet_path)
    text_and_bert = pd.read_parquet(text_and_bert_path)
    train = pd.read_parquet(train_path)

    return attributes, resnet, text_and_bert, train

In [ ]:
# Загрузка трейна
path_attr_train = './data/train/attributes.parquet'
#path_attr_train = 'attr_proc.parquet'
path_res_train = './data/train/resnet.parquet'
path_textbert_train = './data/train/text_and_bert.parquet'
path_id_train= './data/train/train.parquet'
attributes_train, resnet_train, text_and_bert_train, train = load_data(path_attr_train,path_res_train, path_textbert_train, path_id_train)

In [ ]:
DEFAULT_CATEGORY = "Unknown"

class Categories(Struct):
    first: str = mfield(name="1", default_factory=lambda: DEFAULT_CATEGORY)
    second: str = mfield(name="2", default_factory=lambda: DEFAULT_CATEGORY)
    third: str = mfield(name="3", default_factory=lambda: DEFAULT_CATEGORY)
    fourth: str = mfield(name="4", default_factory=lambda: DEFAULT_CATEGORY)


def decode_categories(c: str) -> list[str]:
    categories = decode(c, type=Categories, strict=False)
    return [categories.first, categories.second, categories.third, categories.fourth]

def decode_attributes(a: str) -> str:
    attribute_dict = decode(a)
    return " ".join(starmap(lambda k, v: f"{k} {' '.join(v)}", attribute_dict.items()))

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nonword_regex = re.compile(r"[^\w+]")
russian_stopwords = stopwords.words("russian")

In [ ]:
nonword_regex = re.compile(r"[^\w+]")
russian_stopwords = stopwords.words("russian")
def extract_text_from_row(row):

    # category_text = " ".join(row["categories"])
    attributes_text = row["characteristic_attributes_mapping"]
    # return f"{category_text} {attributes_text}"
    return attributes_text




def clear_description(description: str) -> str:
    return " ".join(
        filter(
            lambda w: w not in russian_stopwords,
            word_tokenize(nonword_regex.sub(" ", description.lower())),
        )
    )



In [ ]:
#Тут уже готовые энкодеры использую

import joblib
from sklearn.preprocessing import LabelEncoder

encoders_path = "category_encoder_%s.pkl"

def get_encoders_paths():
    return map(lambda i: encoders_path % i, range(4))

# Функция для загрузки готовых энкодеров
def load_encoders():
    encoders = []
    for encoder_path in get_encoders_paths():
        encoders.append(joblib.load(encoder_path))
    return encoders

# Функция для обработки категориальных данных и атрибутов
def process_cat_and_attr(df, encoders=[]):
    df["categories"] = df["categories"].apply(decode_categories)
    for i in range(4):
        df[f"category_{i}"] = df["categories"].apply(lambda c: c[i])

    # Используем уже загруженные энкодеры
    for i, e in enumerate(encoders):
        df[f"category_{i}"] = df[f"category_{i}"].apply(
            lambda c: next(iter(e.transform((c,)) or [0]))
        )

    df["characteristic_attributes_mapping"] = df["characteristic_attributes_mapping"].apply(decode_attributes)
    df.drop("categories", axis=1, inplace=True)
    df["combined_text"] = df.apply(extract_text_from_row, axis=1)
    df["combined_text"] = df["combined_text"].fillna(" ").apply(clear_description)
    return df




In [ ]:
# Загрузка готовых энкодеров
encoders = load_encoders()

# Применение энкодеров к новому датафрейму (например, к test dataset)
attributes_train = process_cat_and_attr(attributes_train, encoders=encoders)

# Теперь attributes_test_processed содержит обработанные данные

In [ ]:
attributes_train.head()

,variantid,characteristic_attributes_mapping,category_0,category_1,category_2,category_3,combined_text
0,47920382,Цвет товара бежевый светло-розовый Пол ребенка...,0,8,190,992,цвет товара бежевый светло розовый пол ребенка...
1,49801845,"Количество в упаковке, шт 1 Бренд Нет бренда Т...",0,22,273,2948,количество упаковке шт 1 бренд бренда тип стра...
2,49853444,Бренд Vervaco Тип Набор для вышивания Страна-и...,0,22,313,1847,бренд vervaco тип набор вышивания страна изгот...
3,49893028,"Цвет товара серый Ширина, см 0.8 Бренд Prym Ти...",0,22,323,3148,цвет товара серый ширина см 0 8 бренд prym тип...
4,49987483,Цвет товара разноцветный Название цвета мульти...,0,22,321,1970,цвет товара разноцветный название цвета мульти...


In [ ]:
attributes_train.to_parquet('attr_proc_ind.parquet')

In [ ]:
def prepare_text_and_bert(df: pd.DataFrame) -> pd.DataFrame:
    nltk.download("stopwords")
    nltk.download("punkt_tab")
    df["descritption"] = df["description"].fillna("").apply(clear_description)
    return df
text_and_bert_train = prepare_text_and_bert(text_and_bert_train)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# Удаление дублей
def delete_duples(df):
    df["sorted_variants"] = df[["variantid1", "variantid2"]].apply(
        lambda x: tuple(sorted(x)), axis=1
    )

    # Группируем по столбцу sorted_variants и нахождение групп дубликатов
    duplicate_groups = df[df.duplicated("sorted_variants", keep=False)]

    # Группируем строки по дубликатам и сохраняем индексы и значения target
    grouped_duplicates = duplicate_groups.groupby("sorted_variants").apply(
        lambda x: {"indices": list(x.index), "targets": x["target"].tolist()}
    )

    # Фильтруем группы, где значения target отличаются
    indices_to_drop = []
    for group, info in grouped_duplicates.items():
        targets = info["targets"]
        if (
            len(set(targets)) > 1
        ):  # Если в группе есть более одного уникального значения target
            indices_to_drop.extend(info["indices"])

    # Удаляем строки с различными target и дубликаты из df
    df.drop(indices_to_drop, inplace=True)
    df.drop_duplicates(subset="sorted_variants", keep="first", inplace=True)

    # Удаляем вспомогательный столбец
    df.drop(columns=["sorted_variants"], inplace=True)

    # Обновленный df
    df.reset_index(drop=True, inplace=True)
    return df


def merge_data(train, resnet, attributes, text_and_bert):
    train_data = train.merge(
        resnet[["variantid", "main_pic_embeddings_resnet_v1"]],
        left_on="variantid1",
        right_on="variantid",
        how="left",
    )
    train_data = train_data.rename(
        columns={"main_pic_embeddings_resnet_v1": "pic_embeddings_1"}
    )
    train_data = train_data.drop(columns=["variantid"])

    train_data = train_data.merge(
        resnet[["variantid", "main_pic_embeddings_resnet_v1"]],
        left_on="variantid2",
        right_on="variantid",
        how="left",
    )
    train_data = train_data.rename(
        columns={"main_pic_embeddings_resnet_v1": "pic_embeddings_2"}
    )
    train_data = train_data.drop(columns=["variantid"])

    train_data = train_data.merge(
        attributes[["variantid", "combined_text","category_0", "category_1","category_2", "category_3"]],
        left_on="variantid1",
        right_on="variantid",
        how="left",
    )
    train_data = train_data.rename(columns={"combined_text": "text_1", "category_0": "cat0_1",
                                             "category_1":"cat1_1","category_2": "cat2_1", "category_3": "cat_3_1"})
    train_data = train_data.drop(columns=["variantid"])


    train_data = train_data.merge(
        attributes[["variantid", "combined_text","category_0", "category_1","category_2", "category_3"]],
        left_on="variantid2",
        right_on="variantid",
        how="left",
    )
    train_data = train_data.rename(columns={"combined_text": "text_2","category_0": "cat0_2",
                                             "category_1":"cat1_2","category_2": "cat2_2", "category_3": "cat_3_2"})
    train_data = train_data.drop(columns=["variantid"])
    train_data = train_data.merge(
        text_and_bert[["variantid", "name_bert_64", "description"]],
        left_on="variantid1",
        right_on="variantid",
        how="left",
    )
    train_data = train_data.rename(
        columns={"name_bert_64": "name_1", "description": "description_1"}
    )
    train_data = train_data.drop(columns=["variantid"])
    train_data = train_data.merge(
        text_and_bert[["variantid", "name_bert_64", "description"]],
        left_on="variantid2",
        right_on="variantid",
        how="left",
    )
    train_data = train_data.rename(
        columns={"name_bert_64": "name_2", "description": "description_2"}
    )

    train_data = train_data.drop(columns=["variantid"])
    train_data = delete_duples(train_data)
    train_data.info()
    train_data=train_data.dropna()
    return train_data


In [ ]:
    train_data = merge_data(train, resnet_train, attributes_train, text_and_bert_train)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1163396 entries, 0 to 1163395
Data columns (total 19 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   variantid1        1163396 non-null  int64 
 1   variantid2        1163396 non-null  int64 
 2   target            1163396 non-null  int64 
 3   pic_embeddings_1  1163396 non-null  object
 4   pic_embeddings_2  1163396 non-null  object
 5   text_1            1163396 non-null  object
 6   cat0_1            1163396 non-null  int64 
 7   cat1_1            1163396 non-null  int64 
 8   cat2_1            1163396 non-null  int64 
 9   cat_3_1           1163396 non-null  int64 
 10  text_2            1163396 non-null  object
 11  cat0_2            1163396 non-null  int64 
 12  cat1_2            1163396 non-null  int64 
 13  cat2_2            1163396 non-null  int64 
 14  cat_3_2           1163396 non-null  int64 
 15  name_1            1163396 non-null  object
 16  description_1     

In [ ]:
train_data.to_parquet('train.parquet', index=False)

NameError: name 'train_data' is not defined

In [ ]:
#START HERE With PREPARED TRAIN!!!!!!!!!!!!!!!!

In [ ]:
train_data=pd.read_parquet('train.parquet')

In [ ]:
train_data.head()

,variantid1,variantid2,target,pic_embeddings_1,pic_embeddings_2,text_1,cat0_1,cat1_1,cat2_1,cat_3_1,text_2,cat0_2,cat1_2,cat2_2,cat_3_2,name_1,description_1,name_2,description_2,text_similarity
0,1447875869,1447872068,1,"[[0.5318107604980469, 0.35363996028900146, -0....","[[0.5318107604980469, 0.35363996028900146, -0....",серия одежде обуви harper s bazaar журнал моде...,3,15,336,2741,серия одежде обуви harper s bazaar журнал моде...,3,15,336,2741,"[-0.3927455246448517, 0.4909455478191376, 0.56...",None,"[-0.3927455246448517, 0.4909455478191376, 0.56...",None,1.000000
1,1176231201,284733670,1,"[[0.4308440089225769, 0.7620932459831238, 0.79...","[[0.5668608546257019, 0.9573432803153992, 1.01...",тип фиксатора бдсм наручники вибрация вибрации...,0,19,91,1955,тип фиксатора бдсм оковы вибрация вибрации еди...,0,19,91,1955,"[-0.5404430627822876, 0.31526750326156616, 0.3...",Фиксатор поможет правильно зафиксировать партн...,"[-0.5957552790641785, 0.3414252698421478, 0.46...","БДСМ, для женщин, стандартный, черный <br>мате...",0.751546
2,658617865,549848659,0,"[[-0.36238163709640503, 0.4316844344139099, -0...","[[-0.25123998522758484, 0.3757574260234833, -0...",артикул 83809 комплектация сухой корм 300 г ос...,0,20,238,1345,артикул 10039 единиц одном товаре 1 упаковка п...,0,20,238,1345,"[-0.4767049252986908, 0.5492467880249023, 0.41...",Сухой корм Hill&#39;s Science Plan для взрослы...,"[-0.26443710923194885, 0.3921107053756714, 0.5...",\nСухой корм для взрослых кошек HILL'S SCIENCE...,0.235198
3,719320625,719370486,1,"[[0.7327960729598999, -0.7488707900047302, 0.5...","[[0.7327960729598999, -0.7488707900047302, 0.5...",комплектация бур 310 мм 1 штука цвет товара се...,0,18,413,454,цвет товара серебристый комплектация бур 310 м...,0,18,413,454,"[-0.6116019487380981, 0.4067917764186859, 0.54...",Бур Elitech используется в перфораторах с патр...,"[-0.6116019487380981, 0.4067917764186859, 0.54...",Бур Elitech используется в перфораторах с патр...,1.000000
4,1067645658,949954740,0,"[[-1.3140270709991455, -0.8071212768554688, 0....","[[-0.49589139223098755, -0.5760805606842041, 0...",комплектация товар поставляется упаковке назва...,0,10,366,2197,цвет товара розовый бренд бренда тип пенал стр...,0,10,366,2197,"[-0.6164239048957825, 0.6769619584083557, 0.73...","Материал: Металл, пластик, текстиль<br/> Габар...","[-0.5198364853858948, 0.6435920596122742, 0.64...","Пенал-тубус мягкий 65 х 210 мм, ПТ-22 Милая панда",0.484998


In [ ]:
np.array([train_data.loc[0, "cat0_1"]])

array([3])

In [ ]:
tfidf_vectorizer_text = TfidfVectorizer(max_features=1500)
tfidf_vectorizer_text.fit(train_data['text_1'].tolist() + train_data['text_2'].tolist())

# Сохраняем обученный векторизатор
joblib.dump(tfidf_vectorizer_text, "vectorizer_text.pkl")


['vectorizer_text.pkl']

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
def calculate_similarity(row, column1, column2, tfidf_vectorizer):
    # Преобразуем текстовые данные в вектора с помощью уже обученного векторизатора
    text_embeddings = tfidf_vectorizer.transform([row[column1], row[column2]]).toarray()

    # Рассчитываем косинусное сходство между двумя векторами
    return cosine_similarity([text_embeddings[0]], [text_embeddings[1]])[0][0]

In [ ]:

# Применение функции к колонкам 'text_1' и 'text_2'
train_data['text_similarity'] = train_data.apply(calculate_similarity, axis=1, args=('text_1', 'text_2',tfidf_vectorizer_text))



In [ ]:
train_data.to_parquet('train_full.parquet')

In [ ]:

tfidf_vectorizer_description = joblib.load('vectorizer_description100_cos.pkl')

train_data['description_similarity'] = train_data.apply(calculate_similarity, axis=1, args=('description_1', 'description_2', tfidf_vectorizer_description))



In [ ]:
train_data['description_1'] = train_data['description_1'].str.replace(r'[^\w\s]', '', regex=True)
train_data['description_2'] = train_data['description_2'].str.replace(r'[^\w\s]', '', regex=True)

In [ ]:
train_data['description_1'] = train_data['description_1'].astype(str)
train_data['description_2'] = train_data['description_2'].astype(str)

In [ ]:
train_data["description_1"] = train_data["description_1"].fillna('пусто')
train_data["description_2"] = train_data["description_2"].fillna('пусто')

In [ ]:
train_data['description_1'] = train_data['description_1'].replace('', 'пусто')
train_data['description_2'] = train_data['description_2'].replace('', 'пусто')

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
train_data['description_1'] = train_data['description_1'].fillna("").apply(clear_description)
train_data['description_2'] = train_data['description_2'].fillna("").apply(clear_description)

In [ ]:
train_data.to_parquet('train_new.parquet', index=False)

In [ ]:
train_data.head(20)

,variantid1,variantid2,target,pic_embeddings_1,pic_embeddings_2,text_1,cat0_1,cat1_1,cat2_1,cat_3_1,...,cat0_2,cat1_2,cat2_2,cat_3_2,name_1,description_1,name_2,description_2,text_similarity,similarity_pic
0,1447875869,1447872068,1,"[[0.5318107604980469, 0.35363996028900146, -0....","[[0.5318107604980469, 0.35363996028900146, -0....",серия одежде обуви harper s bazaar журнал моде...,3,15,336,2741,...,3,15,336,2741,"[-0.3927455246448517, 0.4909455478191376, 0.56...",пусто,"[-0.3927455246448517, 0.4909455478191376, 0.56...",пусто,1.000000,1.000000
1,1176231201,284733670,1,"[[0.4308440089225769, 0.7620932459831238, 0.79...","[[0.5668608546257019, 0.9573432803153992, 1.01...",тип фиксатора бдсм наручники вибрация вибрации...,0,19,91,1955,...,0,19,91,1955,"[-0.5404430627822876, 0.31526750326156616, 0.3...",фиксатор поможет правильно зафиксировать партн...,"[-0.5957552790641785, 0.3414252698421478, 0.46...",бдсм женщин стандартный черный br материал пол...,0.751546,0.922073
2,658617865,549848659,0,"[[-0.36238163709640503, 0.4316844344139099, -0...","[[-0.25123998522758484, 0.3757574260234833, -0...",артикул 83809 комплектация сухой корм 300 г ос...,0,20,238,1345,...,0,20,238,1345,"[-0.4767049252986908, 0.5492467880249023, 0.41...",сухой корм hill 39 s science plan взрослых кош...,"[-0.26443710923194885, 0.3921107053756714, 0.5...",сухой корм взрослых кошек hill s science plan ...,0.235198,0.979019
3,719320625,719370486,1,"[[0.7327960729598999, -0.7488707900047302, 0.5...","[[0.7327960729598999, -0.7488707900047302, 0.5...",комплектация бур 310 мм 1 штука цвет товара се...,0,18,413,454,...,0,18,413,454,"[-0.6116019487380981, 0.4067917764186859, 0.54...",бур elitech используется перфораторах патроном...,"[-0.6116019487380981, 0.4067917764186859, 0.54...",бур elitech используется перфораторах патроном...,1.000000,1.000000
4,1067645658,949954740,0,"[[-1.3140270709991455, -0.8071212768554688, 0....","[[-0.49589139223098755, -0.5760805606842041, 0...",комплектация товар поставляется упаковке назва...,0,10,366,2197,...,0,10,366,2197,"[-0.6164239048957825, 0.6769619584083557, 0.73...",материал металл пластик текстиль br габариты,"[-0.5198364853858948, 0.6435920596122742, 0.64...",пенал тубус мягкий 65 х 210 мм пт 22 милая панда,0.484998,0.793333
5,504121134,304997246,0,"[[-0.42657867074012756, 0.2943481206893921, -0...","[[-0.47832685708999634, 0.5721701383590698, 0....",единиц одном товаре 8 упаковка пакет саше вид ...,0,16,453,832,...,0,16,453,832,"[-0.535676121711731, 0.5610489845275879, 0.477...",турбо дрожжи puriferm xxl это профессиональная...,"[-0.5241715312004089, 0.6239168047904968, 0.52...",турбо дрожжи puriferm xxl это профессиональная...,0.974338,0.943608
6,690909146,1044397832,1,"[[0.09168867766857147, 0.663475751876831, 1.16...","[[0.09168867766857147, 0.663475751876831, 1.16...",артикул 77rp008_master kit партномер артикул п...,0,2,38,2101,...,0,2,38,2101,"[-0.6380323171615601, 0.5193229913711548, 0.08...",пусто,"[-0.5618419051170349, 0.3077075183391571, 0.40...",пусто,0.803731,1.000000
7,691943502,691868704,0,"[[-0.07519393414258957, 0.6104053258895874, -0...","[[-0.07519393414258957, 0.6104053258895874, -0...",конструкция очков ободковые материал оправы оч...,0,4,356,1356,...,0,4,356,1356,"[-0.5312095284461975, 0.41729360818862915, 0.4...",очки футляром магните red sun мод 8623 цвет 3 ...,"[-0.5086309909820557, 0.42460229992866516, 0.4...",очки футляром магните red sun мод 8623 цвет 3 ...,0.987953,1.000000
8,479881093,256110461,1,"[[0.12547796964645386, -0.017604805529117584, ...","[[0.16223976016044617, -0.020142126828432083, ...",артикул 5977658 диаметр см 26 тип тарелка коли...,0,9,477,3079,...,0,9,477,3079,"[-0.3130245506763458, 0.6889984011650085, 0.27...",тарелка пицца гладкая красная глина 26 см br b...,"[-0.27131345868110657, 0.7103903293609619, 0.3...",тарелка понравится любителям вкусной полезной ...,0.765061,0.999675
9,905617449,905617985,0,"[[0.658185601234436, -0.8779840469360352, 0.18...","[[0.9926339983940125

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity_pic(row, column1, column2):
    # Получаем два вектора из строк датафрейма
    vector1 = np.array(row[column1][0])
    vector2 = np.array(row[column2][0])

    # Если один из векторов пуст или содержит только нули
    if np.all(vector1 == 0) or np.all(vector2 == 0):
        return 0.0

    # Косинусное сходство: dot(A, B) / (||A|| * ||B||)
    similarity = cosine_similarity([vector1], [vector2])[0][0]

    return similarity


In [ ]:
train_data['similarity_pic'] = train_data.apply(calculate_similarity_pic, axis=1, args=('pic_embeddings_1', 'pic_embeddings_2'))

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1163396 entries, 0 to 1163395
Data columns (total 21 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   variantid1        1163396 non-null  int64  
 1   variantid2        1163396 non-null  int64  
 2   target            1163396 non-null  int64  
 3   pic_embeddings_1  1163396 non-null  object 
 4   pic_embeddings_2  1163396 non-null  object 
 5   text_1            1163396 non-null  object 
 6   cat0_1            1163396 non-null  int64  
 7   cat1_1            1163396 non-null  int64  
 8   cat2_1            1163396 non-null  int64  
 9   cat_3_1           1163396 non-null  int64  
 10  text_2            1163396 non-null  object 
 11  cat0_2            1163396 non-null  int64  
 12  cat1_2            1163396 non-null  int64  
 13  cat2_2            1163396 non-null  int64  
 14  cat_3_2           1163396 non-null  int64  
 15  name_1            1163396 non-null  object 
 16  

In [ ]:
train_data.to_parquet('train.parquet', index=False)

In [ ]:
def calculate_similarity_name(row, column1, column2):
    # Получаем два вектора из строк датафрейма
    vector1 = np.array(row[column1])
    vector2 = np.array(row[column2])

    # Если один из векторов пуст или содержит только нули
    if np.all(vector1 == 0) or np.all(vector2 == 0):
        return 0.0

    # Косинусное сходство: dot(A, B) / (||A|| * ||B||)
    similarity = cosine_similarity([vector1], [vector2])[0][0]

    return similarity

In [ ]:
train_data['similarity_name'] = train_data.apply(calculate_similarity_name, axis=1, args=('name_1', 'name_2'))

In [ ]:
train_data.head()

,variantid1,variantid2,target,pic_embeddings_1,pic_embeddings_2,text_1,cat0_1,cat1_1,cat2_1,cat_3_1,...,cat1_2,cat2_2,cat_3_2,name_1,description_1,name_2,description_2,text_similarity,similarity_pic,similarity_name
0,1447875869,1447872068,1,"[[0.5318107604980469, 0.35363996028900146, -0....","[[0.5318107604980469, 0.35363996028900146, -0....",серия одежде обуви harper s bazaar журнал моде...,3,15,336,2741,...,15,336,2741,"[-0.3927455246448517, 0.4909455478191376, 0.56...",пусто,"[-0.3927455246448517, 0.4909455478191376, 0.56...",пусто,1.000000,1.000000,1.000000
1,1176231201,284733670,1,"[[0.4308440089225769, 0.7620932459831238, 0.79...","[[0.5668608546257019, 0.9573432803153992, 1.01...",тип фиксатора бдсм наручники вибрация вибрации...,0,19,91,1955,...,19,91,1955,"[-0.5404430627822876, 0.31526750326156616, 0.3...",фиксатор поможет правильно зафиксировать партн...,"[-0.5957552790641785, 0.3414252698421478, 0.46...",бдсм женщин стандартный черный br материал пол...,0.751546,0.922073,0.989477
2,658617865,549848659,0,"[[-0.36238163709640503, 0.4316844344139099, -0...","[[-0.25123998522758484, 0.3757574260234833, -0...",артикул 83809 комплектация сухой корм 300 г ос...,0,20,238,1345,...,20,238,1345,"[-0.4767049252986908, 0.5492467880249023, 0.41...",сухой корм hill 39 s science plan взрослых кош...,"[-0.26443710923194885, 0.3921107053756714, 0.5...",сухой корм взрослых кошек hill s science plan ...,0.235198,0.979019,0.966868
3,719320625,719370486,1,"[[0.7327960729598999, -0.7488707900047302, 0.5...","[[0.7327960729598999, -0.7488707900047302, 0.5...",комплектация бур 310 мм 1 штука цвет товара се...,0,18,413,454,...,18,413,454,"[-0.6116019487380981, 0.4067917764186859, 0.54...",бур elitech используется перфораторах патроном...,"[-0.6116019487380981, 0.4067917764186859, 0.54...",бур elitech используется перфораторах патроном...,1.000000,1.000000,1.000000
4,1067645658,949954740,0,"[[-1.3140270709991455, -0.8071212768554688, 0....","[[-0.49589139223098755, -0.5760805606842041, 0...",комплектация товар поставляется упаковке назва...,0,10,366,2197,...,10,366,2197,"[-0.6164239048957825, 0.6769619584083557, 0.73...",материал металл пластик текстиль br габариты,"[-0.5198364853858948, 0.6435920596122742, 0.64...",пенал тубус мягкий 65 х 210 мм пт 22 милая панда,0.484998,0.793333,0.986480


In [ ]:
train_data.to_parquet('train_new.parquet', index=False)

In [ ]:
from sklearn.decomposition import PCA
def prepare_data(train_data):

    # description_data = train_data["description_1"] + " " + train_data["description_2"]
    # description_embeddings = tfidf_vectorizer_description.fit_transform(
    #     description_data
    # ).toarray()

    # print('check2')
    # pca_description = PCA(n_components=100)
    # description_embeddings_reduced = pca_description.fit_transform(description_embeddings)

    # Объединение всех эмбеддингов
    train_data["combined_embeddings"] = train_data.apply(
        lambda row: np.concatenate(
            [
                np.array([row["cat0_1"]]),  # Оборачиваем скаляры в массивы
                np.array([row["cat0_2"]]),
                np.array([row["cat1_1"]]),
                np.array([row["cat1_2"]]),
                np.array([row["cat2_1"]]),
                np.array([row["cat2_2"]]),
                np.array([row["cat_3_1"]]),
                np.array([row["cat_3_2"]]),
                np.array([row["text_similarity"]]),
                np.array([row["similarity_pic"]]),
                np.array([row["similarity_name"]]),
                np.array([row["description_similarity"]])

            ]
        ), axis=1
    )

    X = np.vstack(train_data["combined_embeddings"].values)

    y = train_data["target"]

    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=24)
    return ( X, y,
          X_train,
          X_val,
          y_train,
          y_val,
         # tfidf_vectorizer_description,
      )


def train_model(X_train, y_train):
    bst = XGBClassifier(
        n_estimators=100,
        max_depth=15,
        learning_rate=0.1,
        objective="binary:logistic",
        #reg_lambda=0.1,
        subsample=0.8,
    )

    bst.fit(X_train, y_train)
    joblib.dump(bst, "baseline_xgboost_cat.pkl")

    return bst

def evaluate_model(model, X_val, y_val):
    y_preds = model.predict(X_val)
    precision, recall, _ = precision_recall_curve(y_val, y_preds)
    prauc = auc(recall, precision)
    print(f"PRAUC: {prauc}")



In [ ]:
( X, y,
  X_train,
  X_val,
  y_train,
  y_val,
) = prepare_data(train_data)

step 4


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Пример датафрейма
data = {'text1': ['Пример первого клевого текста', 'Тестовый текст один'],
        'text2': ['Пример второго текста', 'Тестовый текст два']}
df = pd.DataFrame(data)

# Функция для вычисления косинусного сходства
def compute_similarity(text1, text2):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    return similarity[0][0]

# Применение функции к каждому ряду датафрейма
df['similarity'] = df.apply(lambda row: compute_similarity(row['text1'], row['text2']), axis=1)

print(df)


                           text1                  text2  similarity
0  Пример первого клевого текста  Пример второго текста    0.411207
1            Тестовый текст один     Тестовый текст два    0.503103


In [ ]:
model1 = train_model(X_train, y_train)

In [ ]:
evaluate_model(model1, X_val, y_val)


PRAUC: 0.8987690948881317


In [ ]:

def train_model_new(X_train, y_train):
    bst = XGBClassifier(
        n_estimators=400,
        max_depth=10,
        learning_rate=0.5,
        objective="binary:logistic",
        reg_lambda=1.0,
        subsample=0.9,
    )

    bst.fit(X_train, y_train)
    joblib.dump(bst, "xgboost_weights.pkl")

    return bst

In [ ]:
model = train_model_new(X_train, y_train)

In [ ]:
evaluate_model(model, X_val, y_val)

PRAUC: 0.8929600666096955


In [ ]:


model = train_model_new(X, y)

In [ ]:
def train_model_3(X_train, y_train):
    bst = XGBClassifier(
        n_estimators=200,
        max_depth=15,
        learning_rate=0.1,
        objective="binary:logistic",
        reg_lambda=0.2,
        subsample=0.9,
    )

    bst.fit(X_train, y_train)
    joblib.dump(bst, "baseline_xgboost_3.pkl")

    return bst

In [ ]:
model3 = train_model_3(X_train, y_train)

In [ ]:
evaluate_model(model3, X_val, y_val)

PRAUC: 0.9001384763705534
